prerequisite:

`%pip install word2word`

# translate

In [148]:
from word2word import Word2word
import re
import json

In [4]:
en2fr = Word2word("en", "fr")

In [115]:
eng_sentence = "I am a really big fan of trump even though he might be evil."
fr_sentence = "Je suis un grand fan de Trump même s'il pourrait être maléfique."

words = re.sub(r'[^\w\s]', '', eng_sentence).split(" ")

In [116]:
vocabs = []

for word in words:
    try:
        translations = en2fr(word)
        found = ""
        for translation in translations:
            if fr_sentence.find(translation) != -1:

                found = translation
                break
        if found != "" and found != word:
            voc = [word, found]

            vocabs.append(voc)
    except: pass

In [147]:
en2fr_sentence = eng_sentence
en2fr_sentence = '"' + en2fr_sentence + '"'

fr2en_sentence = fr_sentence
fr2en_sentence = '"' + fr2en_sentence + '"'

for v in vocabs:
    en2fr_sentence = en2fr_sentence.replace(v[0], '<'+v[0]+'>')
    fr2en_sentence = fr2en_sentence.replace(v[1], '<'+v[1]+'>')

for v in vocabs:
    en2fr_sentence = en2fr_sentence.replace('<'+v[0]+'>', '"("'+'","'.join(v)+'")"')
    fr2en_sentence = fr2en_sentence.replace('<'+v[1]+'>', '"("'+'","'.join(v[::-1])+'")"')

en2fr_sentence,fr2en_sentence

('"I am a really big fan of trump even though he "("might","pourrait")" "("be","pourrait")" "("evil","maléfique")"."',
 '"Je suis un grand fan de Trump même s\'il <"("pourrait","might")"> être "("maléfique","evil")"."')

In [164]:
en2fr("might")

['pensé', 'pourrais', 'pu', 'aurait', 'pourrait']

In [134]:
fr_sentence

"Je suis un grand fan de Trump même s'il pourrait être maléfique."

# sentence matching

In [ ]:
class w2w_translat

In [154]:
en2de = Word2word("en", "de")

In [ ]:
en2de.

In [160]:
en2de("brunch")

['Frühstück', 'Sonntag', 'beim', 'Morgen', 'hatten']

In [233]:
prince = json.load(open("../static/books/the_little_prince.json"))
ger_prince = json.load(open("../static/books/the_little_prince.ger.json"))

In [234]:
a = prince[0]
b = ger_prince[0]

In [235]:
a

'Once when I was six years old I saw a magnificent picture in a book, called True Stories from  Nature, about the primeval forest.'

In [236]:
b

'Als ich sechs war, sah ich einmal ein wunderbares Bild in einem Buch über den Dschungel, das "Wahre Geschichten" hieß.'

In [219]:
def get_words(sentence):
    words = re.sub(r'[^\w\s]', '', sentence).replace("\n"," ").split(" ")
    words = filter(lambda x: x != "", words)
    return list(words)

In [252]:
def match_score(a:str, b:str, w2w:Word2word = en2de):
    a_words = get_words(a)
    b_words = get_words(b)

    print (a_words)
    print(b_words)

    n = max(len(a_words),len(b_words))
    c = 0

    for a in a_words:
        translations = [a]
        
        try: translations = w2w(a)
        except: pass

        for t in translations:
            if t in b_words:
                c += 1
                break

    return c/n

match_score(a,b)

['Once', 'when', 'I', 'was', 'six', 'years', 'old', 'I', 'saw', 'a', 'magnificent', 'picture', 'in', 'a', 'book', 'called', 'True', 'Stories', 'from', 'Nature', 'about', 'the', 'primeval', 'forest']
['Als', 'ich', 'sechs', 'war', 'sah', 'ich', 'einmal', 'ein', 'wunderbares', 'Bild', 'in', 'einem', 'Buch', 'über', 'den', 'Dschungel', 'das', 'Wahre', 'Geschichten', 'hieß']


0.375

In [250]:
en2de("Once")

['Einmal', 'Sobald', 'Noch', 'erst', 'einmal']

In [269]:
match_score(prince[1],ger_prince[2])

['It', 'was', 'a', 'picture', 'of', 'a', 'boa', 'constrictor', 'in', 'the', 'act', 'of', 'swallowing', 'an', 'animal', 'In', 'the', 'book', 'it', 'said', 'Boa', 'constrictors', 'swallow', 'their', 'prey', 'whole', 'without', 'chewing', 'it']
['Der', 'kleine', 'Prinz', 'Eine', 'Boa', 'die', 'ein', 'wildes', 'Tier', 'verschlingt', 'In', 'diesem', 'Buch', 'heißt', 'es', 'Boas', 'verschlingen', 'ihre', 'Beute', 'als', 'Ganzes', 'ohne', 'zu', 'kauen']


0.3448275862068966

In [258]:
ger_prince[2]

'Der kleine Prinz – Eine Boa, die ein wildes Tier verschlingt In diesem Buch heißt es: "Boas verschlingen ihre Beute als Ganzes, ohne zu kauen.'

In [263]:
en2de("crossword")

['Kreuzworträtsel', 'gelöst', 'löse', 'Rätsel', 'lösen']